In [ ]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)
ws = Workspace.from_config()

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

rest_endpoint1 = "https://northeurope.api.azureml.ms/pipelines/v1.0/subscriptions/ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9/resourceGroups/LUZHAN/providers/Microsoft.MachineLearningServices/workspaces/luzhan-north-europe/PipelineRuns/PipelineSubmit/e9cfeeae-52db-4c43-b1ba-220a66a5f2cf"

# specify the param when running the pipeline
response = requests.post(rest_endpoint1, 
                         headers=aad_token, 
                         json={"ExperimentName": "Test_Model_Register_Exp",
                               "RunSource": "SDK",
                               "ParameterAssignments": {"Dataset1": "3444e16a-245c-4dec-a175-16890e36c665"}})

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception('Received bad response from the endpoint: {}\n'
                    'Response Code: {}\n'
                    'Headers: {}\n'
                    'Content: {}'.format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

In [ ]:
from azureml.pipeline.core.run import PipelineRun
run = PipelineRun.get(ws, run_id)
run.wait_for_completion()

train_model_run = run.find_step_run(name="azureml://Train Model")[0]

In [ ]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
local_path = './model-dataset'
target_path = 'test/model-data'

port_output = train_model_run.get_output_data('Trained_model')
model_file = port_output.download(local_path, overwrite=True)
datastore.upload(src_dir = local_path, target_path = target_path)

In [ ]:
datastore_path = [(datastore, target_path)]
model_dataset = Dataset.File.from_files(path=datastore_path)
registered_model_dataset = model_dataset.register(workspace=ws, name='registered-model-from-sample1', create_new_version=True)